In [10]:
import sys, warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
from utils import *
from fe import *

import pandas as pd
import numpy as np
import datetime
import collections as col
import scipy as sp

In [13]:
dataset_dir = 'dataset/'

In [14]:
data = pd.read_csv('ETHBTC_1510311420_1540493280.csv')

ohlc_feat        = ohlc_feats(data)
ohlc_cross_feat  = ohlc_cross_feats(data)
vol_feat         = vol_feats(data)
ohlc_vol_feat    = ohlc_vol_feats(data)
num_trades_feat  = num_trades_feats(data)

In [ ]:
all_feat = ohlc_feat.merge(ohlc_cross_feat, on='OPEN_TIME')
all_feat = all_feat.merge(vol_feat, on='OPEN_TIME')
all_feat = all_feat.merge(ohlc_vol_feat, on='OPEN_TIME')
all_feat = all_feat.merge(num_trades_feat, on='OPEN_TIME')

In [24]:
def dump_dataset(data, descr):
    import os
    if os.path.isfile(dataset_dir+'ds_meta.csv'):
        dsMeta = pd.read_csv(dataset_dir+'ds_meta.csv')
        pkl_num = dsMeta['ID'].max() + 1
    else:
        dsMeta = pd.DataFrame({'ID':[], 'SHAPE':[], 'DESCR':[]})
        pkl_num = 1
    
    dsMeta = dsMeta.append({'ID':pkl_num, 'SHAPE':data.shape, 'DESCR':descr}, ignore_index=True)
    data.to_pickle(dataset_dir+'{:04d}'.format(pkl_num)+'.pkl')
    dsMeta.to_csv(dataset_dir+'ds_meta.csv', index=False)

In [50]:
def prepare_dataset(data, indices, targets, xWindowSize, yStride, yWindowSize):
    index  = data.loc[:, indices].values
    target = data.loc[:, targets].values

    all_feat = data.drop(indices+targets, axis=1)
    print(all_feat)
    
    X = np.empty((len(all_feat) - xWindowSize - yStride - yWindowSize + 1, xWindowSize*all_feat.shape[1]), dtype=str)
    Y = np.empty((len(all_feat) - xWindowSize - yStride - yWindowSize + 1, yWindowSize*target.shape[1]), dtype=str)
    
    for i in range(len(X)):
        X[i, :] = all_feat.iloc[i:i+xWindowSize, :].values.flatten()
        Y[i, :] = target[i+xWindowSize+yStride:i+xWindowSize+yStride+yWindowSize].flatten()
    
    colNames = ['FEAT_'+str(i)+'_'+x for i in range(xWindowSize, 0, -1) for x in all_feat.columns]
    colNames = indices + colNames + ['TARGET_'+str(i)+'_'+x for i in range(1, yWindowSize+1, 1) for x in targets]
    index = index[xWindowSize-1:xWindowSize-1+len(X)].reshape(-1, len(indices))
    
    ds = np.hstack((index, X, Y))
    ds = pd.DataFrame(data=ds, columns=colNames)
    
    # dump_dataset(ds, str({'xWinSize':xWinSize, 'yStr':yStr, 'yWinSize':yWinSize, 'indices':indices, 'targets':targets}))
    return ds

In [51]:
df = pd.DataFrame({
    'A':['1','2','3','4','5','6','7','8'], 
    'B':['a','b','c','d','e','f','g','h'], 
    'C':['@', '#', '$', '%', '^', '&', '*', '!'],
    'D':['q','w','e','r','t','y','u','i']})
df

A  B  C  D
0  1  a  @  q
1  2  b  #  w
2  3  c  $  e
3  4  d  %  r
4  5  e  ^  t
5  6  f  &  y
6  7  g  *  u
7  8  h  !  i

In [52]:
prepare_dataset(df, ['A'], ['C', 'D'], 3, 1, 2)

   B
0  a
1  b
2  c
3  d
4  e
5  f
6  g
7  h


A FEAT_3_B FEAT_2_B FEAT_1_B TARGET_1_C TARGET_1_D TARGET_2_C TARGET_2_D
0  3        a        b        c          ^          t          &          y
1  4        b        c        d          &          y          *          u
2  5        c        d        e          *          u          !          i

In [ ]:
# Creating Dataset
np.random.seed(0)

data = df.CLOSE.astype(float).values
diff_data = (data[1:] - data[:-1])/data[:-1]
bd_diff_data = np.zeros(diff_data.shape)
bd_diff_data[diff_data > 0.005] = 1
bd_diff_data[diff_data < -0.005] = -1

X_window_size = 120
Y_window_size = 1
X = np.zeros((len(data) - X_window_size - Y_window_size, X_window_size))
Y = np.zeros((len(data) - X_window_size - Y_window_size, Y_window_size))
W = np.zeros((len(data) - X_window_size - Y_window_size, Y_window_size))

for i in range(X.shape[0]):
    X[i] = data[i:i+X_window_size]
    Y[i] = bd_diff_data[i+X_window_size:i+X_window_size+Y_window_size]
    W[i] = diff_data[i+X_window_size:i+X_window_size+Y_window_size]

# sample_zero = list(np.random.choice(np.where(Y == 0)[0], 10000))
sample_pos = list(np.where(Y == 1)[0])
sample_neg = list(np.where(Y == -1)[0])
# sample = sample_zero + sample_pos + sample_neg
sample = sample_pos + sample_neg
sample = sorted(sample)
X = X[sample]
Y = np.ravel(Y[sample])
W = np.ravel(W[sample])

cv_fold_size = 2000
cv_folds = np.random.randint(X.shape[0]-cv_fold_size, size=(10,))